In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
import torch
from x_transformers import ViTransformerWrapper, Encoder
from fastprogress.fastprogress import master_bar, progress_bar
from torchvision.transforms import RandomResizedCrop

/home/dgj335/.local/lib/python3.10/site-packages/beartype/_util/error/utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/home/dgj335/.local/lib/python3.10/site-packages/google/protobuf/internal/__init__.py)
  warn(message, cls)
/home/dgj335/.local/lib/python3.10/site-packages/beartype/_util/error/utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/home/dgj335/.local/lib/python3.10/site-packages/google/protobuf/internal/__init__.py)
  warn(message, cls)
/home/dgj335/.local/lib/python3.10/site-packages/beartype/_util/error/utilerrwarn.py:67: BeartypeModuleUnimportableWarning: Ignoring module "onnx" importation exception:
    ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/home/dgj335/.local/lib/python3.10/si

In [2]:
train_dataset = load_dataset("frgfm/imagenette","full_size",split="train")
rgb_train = train_dataset.filter(lambda s: s["image"].mode == "RGB").with_format("torch")

In [3]:
model = ViTransformerWrapper(
    image_size = 256,
    patch_size = 32,
    num_classes = 10,
    attn_layers = Encoder(
        dim = 512,
        depth = 6,
        heads = 8,
    )
)

In [5]:
epochs = 5
mb = master_bar(range(1, epochs + 1))
train_loss = []
test_loss = []
val_accuracy = []
batch_size = 32
rr_crop = RandomResizedCrop(256,antialias=True);
for epoch in mb:
    dataloader_train = iter(DataLoader(rgb_train, batch_size=1, shuffle=True, drop_last=True))
    model.train()
    running_loss = 0.0
    for i_batch in progress_bar(range(train_dataset.num_rows//batch_size), parent=mb):
        x = torch.zeros(batch_size, 3, 256, 256, dtype=torch.float32)
        targets = torch.zeros(batch_size,dtype=torch.int64)
        for i_sample in range(batch_size):
            inputs = next(dataloader_train)
            x[i_sample,:,:,:] = rr_crop(inputs['image'].permute((0,3,1,2)))
            targets[i_sample] = inputs['label']
        break
        # x = x.cuda()
        # targets = inputs['label'].to(torch.int64).cuda()
    #     optimizer.zero_grad()
    #     outputs = model(x)
    #     loss = loss_function(outputs, targets)
    #     if torch.isnan(loss):
    #         print(f"Loss is nan, skipping batch")
    #         continue
    #     loss.backward()
    #     print(loss.item(), end='\r')
    #     # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    #     optimizer.step()
    #     running_loss += loss.item()

    # train_epoch_loss = running_loss / len(dataloader_train)
    # train_loss.append(train_epoch_loss)
    
    # # Validation Phase
    # dataloader_val = DataLoader(dataset['validation'], batch_size=256, shuffle=False, drop_last=False)
    # model.eval()
    # running_loss_val = 0.0
    # correct_predictions = 0
    # total_predictions = 0
    # with torch.no_grad():
    #     for inputs in dataloader_val:
    #         x = center_crop(inputs['latent'].cuda())
    #         targets = inputs['label'].to(torch.int64).cuda()
    #         outputs = model(x)
    #         loss = loss_function(outputs, targets)
    #         if torch.isnan(loss):
    #             print(f"Loss is nan, skipping batch")
    #             continue
    #         running_loss_val += loss.item()
    #         _, predicted = torch.max(outputs, 1)
    #         correct_predictions += (predicted == targets).sum().item()
    #         total_predictions += targets.size(0)
    # val_epoch_loss = running_loss_val / len(dataloader_val)
    # test_loss.append(val_epoch_loss)
    # epoch_accuracy = correct_predictions / total_predictions
    # print(f'Epoch: {epoch}, Accuracy: {epoch_accuracy:.4f}')
    # val_accuracy.append(epoch_accuracy)
    # graphs = [
    #     [range(1, epoch + 1), train_loss],  # Training Loss
    #     [range(1, epoch + 1), test_loss],  # Validation Loss
    # ]
    # x_bounds = [1, epoch]
    # y_bounds = [min(min(train_loss + test_loss) - 0.05, 0), max(max(train_loss + test_loss) + 0.05, 1)]
    # mb.update_graph(graphs, x_bounds, y_bounds)
    # checkpoint_filename = f'checkpoint{epoch}.pt'
    # torch.save(model.state_dict(), checkpoint_filename)